# 사전 작업

In [1]:
# 데이터 불러오기

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

data = pd.concat([pd.read_csv('./drive/MyDrive/Colab Notebooks/Tennis/1_data_charting-m-points-2020s.csv', encoding='ISO-8859-1'), pd.read_csv('./drive/MyDrive/Colab Notebooks/Tennis/1_data_charting-m-points-2010s.csv', encoding='ISO-8859-1')])

<ipython-input-1-d1f3b1ce96f3>:9: DtypeWarning: Columns (8,9,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat([pd.read_csv('./drive/MyDrive/Colab Notebooks/Tennis/1_data_charting-m-points-2020s.csv', encoding='ISO-8859-1'), pd.read_csv('./drive/MyDrive/Colab Notebooks/Tennis/1_data_charting-m-points-2010s.csv', encoding='ISO-8859-1')])
<ipython-input-1-d1f3b1ce96f3>:9: DtypeWarning: Columns (8,9,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat([pd.read_csv('./drive/MyDrive/Colab Notebooks/Tennis/1_data_charting-m-points-2020s.csv', encoding='ISO-8859-1'), pd.read_csv('./drive/MyDrive/Colab Notebooks/Tennis/1_data_charting-m-points-2010s.csv', encoding='ISO-8859-1')])


## 5세트 대회 필터

In [2]:
# 4대 메이저 대회 필터링 + 5세트 이상까지 간 경기 필터링

match_name = []

for i in data["match_id"].drop_duplicates().values:
    if "US_Open" in i or "Roland_Garros" in i or "Wimbledon" in i or "Australian_Open" in i:
        match_name.append(i)

data = data[data["match_id"].isin(match_name)]

tmp = data.groupby(["match_id"])[["Set1","Set2"]].max()
cond1 = tmp[tmp["Set1"]+tmp["Set2"]==4].reset_index()["match_id"]
data = data.merge(cond1, on="match_id", how="inner")



# 대회명 형식 변경

data['match_id'] = data['match_id'].str.split('-').str[-2:]
data['match_id'] = data['match_id'].astype(str)
data_2 = data.copy()
data_2['match_id'] = data_2['match_id'].str.strip('[]').str.split(',').str[1]
data['match_id'] = data['match_id'].str.strip('[]').str.split(',').str[0]

data_3 = data.copy()
data_4 = data_2.copy()

## 3세트(3-0) 대회 필터

In [ ]:
# 3-0 대회 필터

match_name = []

for i in data["match_id"].drop_duplicates().values:
    if "US_Open" in i or "Roland_Garros" in i or "Wimbledon" in i or "Australian_Open" in i:
        match_name.append(i)

data = data[data["match_id"].isin(match_name)]

tmp = data.groupby(["match_id"])[["Set1", "Set2"]].max()
cond1 = tmp[(tmp["Set1"] == 2) & (tmp["Set2"] == 0) | (tmp["Set1"] == 0) & (tmp["Set2"] == 2)].reset_index()["match_id"]
data = data.merge(cond1, on="match_id", how="inner")



# 대회명 형식 변경

data['match_id'] = data['match_id'].str.split('-').str[-2:]
data['match_id'] = data['match_id'].astype(str)
data_2 = data.copy()
data_2['match_id'] = data_2['match_id'].str.strip('[]').str.split(',').str[1]
data['match_id'] = data['match_id'].str.strip('[]').str.split(',').str[0]

data_3 = data.copy()
data_4 = data_2.copy()

## ---

In [3]:
# 서버 데이터 추출

data_server = data.loc[(data['Svr'] == 1), :]
data_2_server = data_2.loc[(data_2['Svr'] == 2), :]

server = pd.concat([data_server,data_2_server], join='outer', axis=0, ignore_index=True)
server['2ndIn'] = server['2ndIn'].fillna(0)



# 리시버 데이터 추출

data_receiver = data_3.loc[(data_3['Svr'] == 2), :]
data_2_receiver = data_4.loc[(data_4['Svr'] == 1), :]

receiver = pd.concat([data_receiver,data_2_receiver], join='outer', axis=0, ignore_index=True)
receiver['isRcvrWinner'] = 1 - receiver['isSvrWinner']
receiver['2ndIn'] = receiver['2ndIn'].fillna(0)

In [4]:
# 그룹화

server_group_1 = server[server['Pts'].isin(['40-0', '40-15'])].copy()
server_group_2 = server[server['Pts'].isin(['40-30', 'AD-40'])].copy()
server_group_12 = server[server['Pts'].isin(['40-0', '40-15', '40-30', 'AD-40'])].copy()

server_group_3 = server[server['Pts'].isin(['0-40', '15-40'])].copy()
server_group_4 = server[server['Pts'].isin(['30-40', '40-AD'])].copy()
server_group_34 = server[server['Pts'].isin(['0-40', '15-40', '30-40', '40-AD'])].copy()



receiver_group_1 = receiver[receiver['Pts'].isin(['40-0', '40-15'])].copy()
receiver_group_2 = receiver[receiver['Pts'].isin(['40-30', 'AD-40'])].copy()
receiver_group_12 = receiver[receiver['Pts'].isin(['40-0', '40-15', '40-30', 'AD-40'])].copy()

receiver_group_3 = receiver[receiver['Pts'].isin(['0-40', '15-40'])].copy()
receiver_group_4 = receiver[receiver['Pts'].isin(['30-40', '40-AD'])].copy()
receiver_group_34 = receiver[receiver['Pts'].isin(['0-40', '15-40', '30-40', '40-AD'])].copy()

# 득점률

### Server용

In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1)
n_2 = len(server_group_2)
n_3 = len(server_group_3)
n_4 = len(server_group_4)
n_12 = len(server_group_12)
n_34 = len(server_group_34)

x_1 = len(server_group_1[server_group_1['isSvrWinner']==1])
x_2 = len(server_group_2[server_group_2['isSvrWinner']==1])
x_3 = len(server_group_3[server_group_3['isSvrWinner']==1])
x_4 = len(server_group_4[server_group_4['isSvrWinner']==1])
x_12 = len(server_group_12[server_group_12['isSvrWinner']==1])
x_34 = len(server_group_34[server_group_34['isSvrWinner']==1])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.6743337563451777

In [ ]:
p_2

0.6339079438421894

In [ ]:
p_12

0.6552677897913

In [ ]:
p_3

0.6330567964262923

In [ ]:
p_4

0.60928

In [ ]:
p_34

0.6172208013640239

### Receiver용

In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(receiver_group_1)
n_2 = len(receiver_group_2)
n_3 = len(receiver_group_3)
n_4 = len(receiver_group_4)
n_12 = len(receiver_group_12)
n_34 = len(receiver_group_34)

x_1 = len(receiver_group_1[receiver_group_1['isRcvrWinner']==1])
x_2 = len(receiver_group_2[receiver_group_2['isRcvrWinner']==1])
x_3 = len(receiver_group_3[receiver_group_3['isRcvrWinner']==1])
x_4 = len(receiver_group_4[receiver_group_4['isRcvrWinner']==1])
x_12 = len(receiver_group_12[receiver_group_12['isRcvrWinner']==1])
x_34 = len(receiver_group_34[receiver_group_34['isRcvrWinner']==1])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.32566624365482233

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 4692
게임포인트의 n: 11931
 
브레이크포인트의 x: 1796
게임포인트의 x: 4113
 
브레이크포인트의 p: 0.38277919863597615
게임포인트의 p: 0.34473221020870004
 
z_score: 4.571
p_value: 2.428998182812947e-06
 
가설 채택


## 0-40, 15-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_1 * (1 - p_1) / n_1)
z_score = (p_3 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 1567
40-0, 40-15의 n: 6304
 
0-40, 15-40의 x: 575
40-0, 40-15의 x: 2053
 
0-40, 15-40의 p: 0.36694320357370774
40-0, 40-15의 p: 0.32566624365482233
 
z_score: 3.051
p_value: 0.0011418283306632304
 
가설 채택


## 30-40, 40-AD의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_1 * (1 - p_1) / n_1)
z_score = (p_4 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 3125
40-0, 40-15의 n: 6304
 
30-40, 40-AD의 x: 1221
40-0, 40-15의 x: 2053
 
30-40, 40-AD의 p: 0.39072
40-0, 40-15의 p: 0.32566624365482233
 
z_score: 6.174
p_value: 3.3245795005854006e-10
 
가설 채택


## 0-40, 15-40의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_2 * (1 - p_2) / n_2)
z_score = (p_3 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 1567
40-30, AD-40의 n: 5627
 
0-40, 15-40의 x: 575
40-30, AD-40의 x: 2060
 
0-40, 15-40의 p: 0.36694320357370774
40-30, AD-40의 p: 0.36609205615781054
 
z_score: 0.062
p_value: 0.47534804127004804
 
가설 기각


## 30-40, 40-AD의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_2 * (1 - p_2) / n_2)
z_score = (p_4 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 3125
40-30, AD-40의 n: 5627
 
30-40, 40-AD의 x: 1221
40-30, AD-40의 x: 2060
 
30-40, 40-AD의 p: 0.39072
40-30, AD-40의 p: 0.36609205615781054
 
z_score: 2.273
p_value: 0.011519913613886268
 
가설 채택


## 40-30, AD-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("40-30, AD-40의 n:", n_2)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("40-30, AD-40의 x:", x_2)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("40-30, AD-40의 p:", p_2)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_2 * (1 - p_2) / n_2 + p_1 * (1 - p_1) / n_1)
z_score = (p_2 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

40-30, AD-40의 n: 5627
40-0, 40-15의 n: 6304
 
40-30, AD-40의 x: 2060
40-0, 40-15의 x: 2053
 
40-30, AD-40의 p: 0.36609205615781054
40-0, 40-15의 p: 0.32566624365482233
 
z_score: 4.635
p_value: 1.786641484868845e-06
 
가설 채택


## 30-40, 40-AD의 상황 - 0-40, 15-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("0-40, 15-40의 n:", n_3)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("0-40, 15-40의 x:", x_3)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("0-40, 15-40의 p:", p_3)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_3 * (1 - p_3) / n_3)
z_score = (p_4 - p_3) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 3125
0-40, 15-40의 n: 1567
 
30-40, 40-AD의 x: 1221
0-40, 15-40의 x: 575
 
30-40, 40-AD의 p: 0.39072
0-40, 15-40의 p: 0.36694320357370774
 
z_score: 1.587
p_value: 0.05623782154521295
 
가설 기각


---

# 에이스

In [ ]:
server_group_1['isAce_label'] = pd.Categorical(server_group_1['isAce']).codes
server_group_2['isAce_label'] = pd.Categorical(server_group_2['isAce']).codes
server_group_12['isAce_label'] = pd.Categorical(server_group_12['isAce']).codes

server_group_3['isAce_label'] = pd.Categorical(server_group_3['isAce']).codes
server_group_4['isAce_label'] = pd.Categorical(server_group_4['isAce']).codes
server_group_34['isAce_label'] = pd.Categorical(server_group_34['isAce']).codes

### Server용

In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1)
n_2 = len(server_group_2)
n_3 = len(server_group_3)
n_4 = len(server_group_4)
n_12 = len(server_group_12)
n_34 = len(server_group_34)

x_1 = len(server_group_1[server_group_1['isAce_label']==1])
x_2 = len(server_group_2[server_group_2['isAce_label']==1])
x_3 = len(server_group_3[server_group_3['isAce_label']==1])
x_4 = len(server_group_4[server_group_4['isAce_label']==1])
x_12 = len(server_group_12[server_group_12['isAce_label']==1])
x_34 = len(server_group_34[server_group_34['isAce_label']==1])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.11960659898477158

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 4692
게임포인트의 n: 11931
 
브레이크포인트의 x: 291
게임포인트의 x: 1201
 
브레이크포인트의 p: 0.062020460358056265
게임포인트의 p: 0.10066214064202497
 
z_score: -8.644
p_value: 0.0
 
가설 채택


## 0-40, 15-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_1 * (1 - p_1) / n_1)
z_score = (p_3 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 1567
40-0, 40-15의 n: 6304
 
0-40, 15-40의 x: 98
40-0, 40-15의 x: 754
 
0-40, 15-40의 p: 0.06253988513082323
40-0, 40-15의 p: 0.11960659898477158
 
z_score: -7.757
p_value: 4.3298697960381105e-15
 
가설 채택


## 30-40, 40-AD의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_1 * (1 - p_1) / n_1)
z_score = (p_4 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 3125
40-0, 40-15의 n: 6304
 
30-40, 40-AD의 x: 193
40-0, 40-15의 x: 754
 
30-40, 40-AD의 p: 0.06176
40-0, 40-15의 p: 0.11960659898477158
 
z_score: -9.744
p_value: 0.0
 
가설 채택


## 0-40, 15-40의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_2 * (1 - p_2) / n_2)
z_score = (p_3 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 1567
40-30, AD-40의 n: 5627
 
0-40, 15-40의 x: 98
40-30, AD-40의 x: 447
 
0-40, 15-40의 p: 0.06253988513082323
40-30, AD-40의 p: 0.07943842189443753
 
z_score: -2.38
p_value: 0.008654812691576086
 
가설 채택


## 30-40, 40-AD의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_2 * (1 - p_2) / n_2)
z_score = (p_4 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 3125
40-30, AD-40의 n: 5627
 
30-40, 40-AD의 x: 193
40-30, AD-40의 x: 447
 
30-40, 40-AD의 p: 0.06176
40-30, AD-40의 p: 0.07943842189443753
 
z_score: -3.148
p_value: 0.0008222021360496656
 
가설 채택


## 40-30, AD-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("40-30, AD-40의 n:", n_2)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("40-30, AD-40의 x:", x_2)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("40-30, AD-40의 p:", p_2)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_2 * (1 - p_2) / n_2 + p_1 * (1 - p_1) / n_1)
z_score = (p_2 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

40-30, AD-40의 n: 5627
40-0, 40-15의 n: 6304
 
40-30, AD-40의 x: 447
40-0, 40-15의 x: 754
 
40-30, AD-40의 p: 0.07943842189443753
40-0, 40-15의 p: 0.11960659898477158
 
z_score: -7.371
p_value: 8.493206138382448e-14
 
가설 채택


## 30-40, 40-AD의 상황 - 0-40, 15-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("0-40, 15-40의 n:", n_3)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("0-40, 15-40의 x:", x_3)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("0-40, 15-40의 p:", p_3)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_3 * (1 - p_3) / n_3)
z_score = (p_4 - p_3) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 3125
0-40, 15-40의 n: 1567
 
30-40, 40-AD의 x: 193
0-40, 15-40의 x: 98
 
30-40, 40-AD의 p: 0.06176
0-40, 15-40의 p: 0.06253988513082323
 
z_score: -0.104
p_value: 0.458482947377582
 
가설 기각


---

# 세컨드 서브 성공률

### Server용

In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1[server_group_1['1stIn']==0])
n_2 = len(server_group_2[server_group_2['1stIn']==0])
n_3 = len(server_group_3[server_group_3['1stIn']==0])
n_4 = len(server_group_4[server_group_4['1stIn']==0])
n_12 = len(server_group_12[server_group_12['1stIn']==0])
n_34 = len(server_group_34[server_group_34['1stIn']==0])

x_1 = len(server_group_1[(server_group_1['1stIn']==0) & (server_group_1['2ndIn']==1)])
x_2 = len(server_group_2[(server_group_2['1stIn']==0) & (server_group_2['2ndIn']==1)])
x_3 = len(server_group_3[(server_group_3['1stIn']==0) & (server_group_3['2ndIn']==1)])
x_4 = len(server_group_4[(server_group_4['1stIn']==0) & (server_group_4['2ndIn']==1)])
x_12 = len(server_group_12[(server_group_12['1stIn']==0) & (server_group_12['2ndIn']==1)])
x_34 = len(server_group_34[(server_group_34['1stIn']==0) & (server_group_34['2ndIn']==1)])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.8992042440318302

In [ ]:
p_2

0.9112576064908722

In [ ]:
p_12

0.9048181388757676

In [ ]:
p_3

0.9017160686427457

In [ ]:
p_4

0.9110032362459547

In [ ]:
p_34

0.9078316462440064

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 1877
게임포인트의 n: 4234
 
브레이크포인트의 x: 1704
게임포인트의 x: 3831
 
브레이크포인트의 p: 0.9078316462440064
게임포인트의 p: 0.9048181388757676
 
z_score: 0.374
p_value: 0.354197302481971
 
가설 기각


## 0-40, 15-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_1 * (1 - p_1) / n_1)
z_score = (p_3 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 641
40-0, 40-15의 n: 2262
 
0-40, 15-40의 x: 578
40-0, 40-15의 x: 2034
 
0-40, 15-40의 p: 0.9017160686427457
40-0, 40-15의 p: 0.8992042440318302
 
z_score: 0.188
p_value: 0.4254007501177418
 
가설 기각


## 30-40, 40-AD의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_1 * (1 - p_1) / n_1)
z_score = (p_4 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 1236
40-0, 40-15의 n: 2262
 
30-40, 40-AD의 x: 1126
40-0, 40-15의 x: 2034
 
30-40, 40-AD의 p: 0.9110032362459547
40-0, 40-15의 p: 0.8992042440318302
 
z_score: 1.148
p_value: 0.1255180726437216
 
가설 기각


## 0-40, 15-40의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_2 * (1 - p_2) / n_2)
z_score = (p_3 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 641
40-30, AD-40의 n: 1972
 
0-40, 15-40의 x: 578
40-30, AD-40의 x: 1797
 
0-40, 15-40의 p: 0.9017160686427457
40-30, AD-40의 p: 0.9112576064908722
 
z_score: -0.713
p_value: 0.2380352771629709
 
가설 기각


## 30-40, 40-AD의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_2 * (1 - p_2) / n_2)
z_score = (p_4 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 1236
40-30, AD-40의 n: 1972
 
30-40, 40-AD의 x: 1126
40-30, AD-40의 x: 1797
 
30-40, 40-AD의 p: 0.9110032362459547
40-30, AD-40의 p: 0.9112576064908722
 
z_score: -0.025
p_value: 0.4901724145723265
 
가설 기각


## 40-30, AD-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("40-30, AD-40의 n:", n_2)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("40-30, AD-40의 x:", x_2)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("40-30, AD-40의 p:", p_2)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_2 * (1 - p_2) / n_2 + p_1 * (1 - p_1) / n_1)
z_score = (p_2 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

40-30, AD-40의 n: 1972
40-0, 40-15의 n: 2262
 
40-30, AD-40의 x: 1797
40-0, 40-15의 x: 2034
 
40-30, AD-40의 p: 0.9112576064908722
40-0, 40-15의 p: 0.8992042440318302
 
z_score: 1.339
p_value: 0.09034569709282936
 
가설 기각


## 30-40, 40-AD의 상황 - 0-40, 15-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("0-40, 15-40의 n:", n_3)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("0-40, 15-40의 x:", x_3)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("0-40, 15-40의 p:", p_3)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_3 * (1 - p_3) / n_3)
z_score = (p_4 - p_3) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 1236
0-40, 15-40의 n: 641
 
30-40, 40-AD의 x: 1126
0-40, 15-40의 x: 578
 
30-40, 40-AD의 p: 0.9110032362459547
0-40, 15-40의 p: 0.9017160686427457
 
z_score: 0.65
p_value: 0.25769661024797696
 
가설 기각


---

# 더블 폴트 확률

### Server용

In [ ]:
server_group_1['isDouble_label'] = pd.Categorical(server_group_1['isDouble']).codes
server_group_2['isDouble_label'] = pd.Categorical(server_group_2['isDouble']).codes
server_group_12['isDouble_label'] = pd.Categorical(server_group_12['isDouble']).codes

server_group_3['isDouble_label'] = pd.Categorical(server_group_3['isDouble']).codes
server_group_4['isDouble_label'] = pd.Categorical(server_group_4['isDouble']).codes
server_group_34['isDouble_label'] = pd.Categorical(server_group_34['isDouble']).codes

In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1[server_group_1['1stIn']==0])
n_2 = len(server_group_2[server_group_2['1stIn']==0])
n_3 = len(server_group_3[server_group_3['1stIn']==0])
n_4 = len(server_group_4[server_group_4['1stIn']==0])
n_12 = len(server_group_12[server_group_12['1stIn']==0])
n_34 = len(server_group_34[server_group_34['1stIn']==0])

x_1 = len(server_group_1[(server_group_1['1stIn']==0) & (server_group_1['isDouble_label']==1)])
x_2 = len(server_group_2[(server_group_2['1stIn']==0) & (server_group_2['isDouble_label']==1)])
x_3 = len(server_group_3[(server_group_3['1stIn']==0) & (server_group_3['isDouble_label']==1)])
x_4 = len(server_group_4[(server_group_4['1stIn']==0) & (server_group_4['isDouble_label']==1)])
x_12 = len(server_group_12[(server_group_12['1stIn']==0) & (server_group_12['isDouble_label']==1)])
x_34 = len(server_group_34[(server_group_34['1stIn']==0) & (server_group_34['isDouble_label']==1)])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.09372236958443855

In [ ]:
p_2

0.08417849898580122

In [ ]:
p_12

0.08927727916863486

In [ ]:
p_3

0.09516380655226209

In [ ]:
p_4

0.08737864077669903

In [ ]:
p_34

0.09003729355354288

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 1877
게임포인트의 n: 4234
 
브레이크포인트의 x: 169
게임포인트의 x: 378
 
브레이크포인트의 p: 0.09003729355354288
게임포인트의 p: 0.08927727916863486
 
z_score: 0.096
p_value: 0.4618140277985079
 
가설 기각


## 0-40, 15-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_1 * (1 - p_1) / n_1)
z_score = (p_3 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 641
40-0, 40-15의 n: 2262
 
0-40, 15-40의 x: 61
40-0, 40-15의 x: 212
 
0-40, 15-40의 p: 0.09516380655226209
40-0, 40-15의 p: 0.09372236958443855
 
z_score: 0.11
p_value: 0.4562261865406725
 
가설 기각


## 30-40, 40-AD의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_1 * (1 - p_1) / n_1)
z_score = (p_4 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 1236
40-0, 40-15의 n: 2262
 
30-40, 40-AD의 x: 108
40-0, 40-15의 x: 212
 
30-40, 40-AD의 p: 0.08737864077669903
40-0, 40-15의 p: 0.09372236958443855
 
z_score: -0.628
p_value: 0.26502996555939917
 
가설 기각


## 0-40, 15-40의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("0-40, 15-40의 n:", n_3)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("0-40, 15-40의 x:", x_3)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("0-40, 15-40의 p:", p_3)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_3 * (1 - p_3) / n_3 + p_2 * (1 - p_2) / n_2)
z_score = (p_3 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

0-40, 15-40의 n: 641
40-30, AD-40의 n: 1972
 
0-40, 15-40의 x: 61
40-30, AD-40의 x: 166
 
0-40, 15-40의 p: 0.09516380655226209
40-30, AD-40의 p: 0.08417849898580122
 
z_score: 0.834
p_value: 0.20209288644665535
 
가설 기각


## 30-40, 40-AD의 상황 - 40-30, AD-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("40-30, AD-40의 n:", n_2)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("40-30, AD-40의 x:", x_2)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("40-30, AD-40의 p:", p_2)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_2 * (1 - p_2) / n_2)
z_score = (p_4 - p_2) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 1236
40-30, AD-40의 n: 1972
 
30-40, 40-AD의 x: 108
40-30, AD-40의 x: 166
 
30-40, 40-AD의 p: 0.08737864077669903
40-30, AD-40의 p: 0.08417849898580122
 
z_score: 0.314
p_value: 0.3766131153706199
 
가설 기각


## 40-30, AD-40의 상황 - 40-0, 40-15의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("40-30, AD-40의 n:", n_2)
print("40-0, 40-15의 n:", n_1)
print(" ")
print("40-30, AD-40의 x:", x_2)
print("40-0, 40-15의 x:", x_1)
print(" ")
print("40-30, AD-40의 p:", p_2)
print("40-0, 40-15의 p:", p_1)
print(" ")

SE = np.sqrt(p_2 * (1 - p_2) / n_2 + p_1 * (1 - p_1) / n_1)
z_score = (p_2 - p_1) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

40-30, AD-40의 n: 1972
40-0, 40-15의 n: 2262
 
40-30, AD-40의 x: 166
40-0, 40-15의 x: 212
 
40-30, AD-40의 p: 0.08417849898580122
40-0, 40-15의 p: 0.09372236958443855
 
z_score: -1.09
p_value: 0.13782354036957267
 
가설 기각


## 30-40, 40-AD의 상황 - 0-40, 15-40의 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("30-40, 40-AD의 n:", n_4)
print("0-40, 15-40의 n:", n_3)
print(" ")
print("30-40, 40-AD의 x:", x_4)
print("0-40, 15-40의 x:", x_3)
print(" ")
print("30-40, 40-AD의 p:", p_4)
print("0-40, 15-40의 p:", p_3)
print(" ")

SE = np.sqrt(p_4 * (1 - p_4) / n_4 + p_3 * (1 - p_3) / n_3)
z_score = (p_4 - p_3) / SE
p_value = 1 - norm.cdf(abs(z_score))

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

30-40, 40-AD의 n: 1236
0-40, 15-40의 n: 641
 
30-40, 40-AD의 x: 108
0-40, 15-40의 x: 61
 
30-40, 40-AD의 p: 0.08737864077669903
0-40, 15-40의 p: 0.09516380655226209
 
z_score: -0.552
p_value: 0.2904458251481209
 
가설 기각


---

# Unforced Error

### Server용

In [ ]:
server_group_1['isUnforced_label'] = pd.Categorical(server_group_1['isUnforced']).codes
server_group_2['isUnforced_label'] = pd.Categorical(server_group_2['isUnforced']).codes
server_group_12['isUnforced_label'] = pd.Categorical(server_group_12['isUnforced']).codes

server_group_3['isUnforced_label'] = pd.Categorical(server_group_3['isUnforced']).codes
server_group_4['isUnforced_label'] = pd.Categorical(server_group_4['isUnforced']).codes
server_group_34['isUnforced_label'] = pd.Categorical(server_group_34['isUnforced']).codes



# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1)
n_2 = len(server_group_2)
n_3 = len(server_group_3)
n_4 = len(server_group_4)
n_12 = len(server_group_12)
n_34 = len(server_group_34)

x_1 = len(server_group_1[(server_group_1['isUnforced_label']==1) & (server_group_1['isSvrWinner']==0)])
x_2 = len(server_group_2[(server_group_2['isUnforced_label']==1) & (server_group_2['isSvrWinner']==0)])
x_3 = len(server_group_3[(server_group_3['isUnforced_label']==1) & (server_group_3['isSvrWinner']==0)])
x_4 = len(server_group_4[(server_group_4['isUnforced_label']==1) & (server_group_4['isSvrWinner']==0)])
x_12 = len(server_group_12[(server_group_12['isUnforced_label']==1) & (server_group_12['isSvrWinner']==0)])
x_34 = len(server_group_34[(server_group_34['isUnforced_label']==1) & (server_group_34['isSvrWinner']==0)])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.125

In [ ]:
p_2

0.15318997689710326

In [ ]:
p_12

0.13829519738496354

In [ ]:
p_3

0.19527760051052967

In [ ]:
p_4

0.19488

In [ ]:
p_34

0.19501278772378516

### Receiver용

In [ ]:
receiver_group_1['isUnforced_label'] = pd.Categorical(receiver_group_1['isUnforced']).codes
receiver_group_2['isUnforced_label'] = pd.Categorical(receiver_group_2['isUnforced']).codes
receiver_group_12['isUnforced_label'] = pd.Categorical(receiver_group_12['isUnforced']).codes

receiver_group_3['isUnforced_label'] = pd.Categorical(receiver_group_3['isUnforced']).codes
receiver_group_4['isUnforced_label'] = pd.Categorical(receiver_group_4['isUnforced']).codes
receiver_group_34['isUnforced_label'] = pd.Categorical(receiver_group_34['isUnforced']).codes


# n, x, p(=x/n) 값 구하기

n_1 = len(receiver_group_1)
n_2 = len(receiver_group_2)
n_3 = len(receiver_group_3)
n_4 = len(receiver_group_4)
n_12 = len(receiver_group_12)
n_34 = len(receiver_group_34)

x_1 = len(receiver_group_1[(receiver_group_1['isUnforced_label']==1) & (receiver_group_1['isRcvrWinner']==0)])
x_2 = len(receiver_group_2[(receiver_group_2['isUnforced_label']==1) & (receiver_group_2['isRcvrWinner']==0)])
x_3 = len(receiver_group_3[(receiver_group_3['isUnforced_label']==1) & (receiver_group_3['isRcvrWinner']==0)])
x_4 = len(receiver_group_4[(receiver_group_4['isUnforced_label']==1) & (receiver_group_4['isRcvrWinner']==0)])
x_12 = len(receiver_group_12[(receiver_group_12['isUnforced_label']==1) & (receiver_group_12['isRcvrWinner']==0)])
x_34 = len(receiver_group_34[(receiver_group_34['isUnforced_label']==1) & (receiver_group_34['isRcvrWinner']==0)])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34



In [ ]:
p_1

0.14546319796954316

In [ ]:
p_2

0.1638528523191754

In [ ]:
p_12

0.15413628363087756

In [ ]:
p_3

0.15890236119974474

In [ ]:
p_4

0.15264

In [ ]:
p_34

0.1547314578005115

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 4692
게임포인트의 n: 11931
 
브레이크포인트의 x: 726
게임포인트의 x: 1839
 
브레이크포인트의 p: 0.1547314578005115
게임포인트의 p: 0.15413628363087756
 
z_score: 0.096
p_value: 0.4619405355243127
 
가설 기각


# Scoring Rate - Rally

### Server용

In [ ]:
server_group_1

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount
8,'Hubert_Hurkacz',14,0,0,0,2.0,40-0,3 (4),1,0,...,1.0,False,False,False,False,True,False,1,1,1
12,'Hubert_Hurkacz',24,0,0,1,3.0,40-0,5 (4),1,0,...,0.0,False,False,False,False,True,False,2,0,2
13,'Hubert_Hurkacz',25,0,0,1,3.0,40-15,5 (5),1,0,...,0.0,False,False,False,True,False,False,2,0,8
19,'Hubert_Hurkacz',37,0,0,2,4.0,40-15,7 (5),1,0,...,0.0,True,False,False,False,False,False,1,1,1
23,'Hubert_Hurkacz',49,0,0,3,5.0,40-0,9 (4),1,0,...,0.0,False,False,True,False,False,False,2,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61399,'Fernando_Gonzalez',188,1,2,2,1.0,40-0,32 (4),1,0,...,0.0,True,False,False,False,False,False,2,1,1.0
61433,'Fernando_Gonzalez',241,1,2,6,5.0,40-0,40 (4),1,0,...,1.0,False,False,False,True,False,False,1,0,2.0
61434,'Fernando_Gonzalez',242,1,2,6,5.0,40-15,40 (5),1,0,...,0.0,False,False,True,False,False,False,1,0,8.0
61447,'Fernando_Gonzalez',266,2,2,3,0.0,40-0,44 (4),0,0,...,0.0,False,False,True,False,False,False,1,0,4.0


In [ ]:
server_group_1 = server_group_1.astype({'rallyCount': 'int64'})
server_group_2 = server_group_2.astype({'rallyCount': 'int64'})
server_group_12 = server_group_12.astype({'rallyCount': 'int64'})

server_group_3 = server_group_3.astype({'rallyCount': 'int64'})
server_group_4 = server_group_4.astype({'rallyCount': 'int64'})
server_group_34 = server_group_34.astype({'rallyCount': 'int64'})



# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1[server_group_1['rallyCount']!=0])
n_2 = len(server_group_2[server_group_2['rallyCount']!=0])
n_3 = len(server_group_3[server_group_3['rallyCount']!=0])
n_4 = len(server_group_4[server_group_4['rallyCount']!=0])
n_12 = len(server_group_12[server_group_12['rallyCount']!=0])
n_34 = len(server_group_34[server_group_34['rallyCount']!=0])

x_1 = len(server_group_1[(server_group_1['rallyCount']!=0) & (server_group_1['isSvrWinner']==1)])
x_2 = len(server_group_2[(server_group_2['rallyCount']!=0) & (server_group_2['isSvrWinner']==1)])
x_3 = len(server_group_3[(server_group_3['rallyCount']!=0) & (server_group_3['isSvrWinner']==1)])
x_4 = len(server_group_4[(server_group_4['rallyCount']!=0) & (server_group_4['isSvrWinner']==1)])
x_12 = len(server_group_12[(server_group_12['rallyCount']!=0) & (server_group_12['isSvrWinner']==1)])
x_34 = len(server_group_34[(server_group_34['rallyCount']!=0) & (server_group_34['isSvrWinner']==1)])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.7024124256444151

In [ ]:
p_2

0.6535360938072554

In [ ]:
p_12

0.6792354474370113

In [ ]:
p_3

0.6504918032786885

In [ ]:
p_4

0.6283828382838283

In [ ]:
p_34

0.6357848518111965

### Receiver용

In [ ]:
receiver_group_1 = receiver_group_1.astype({'rallyCount': 'int64'})
receiver_group_2 = receiver_group_2.astype({'rallyCount': 'int64'})
receiver_group_12 = receiver_group_12.astype({'rallyCount': 'int64'})

receiver_group_3 = receiver_group_3.astype({'rallyCount': 'int64'})
receiver_group_4 = receiver_group_4.astype({'rallyCount': 'int64'})
receiver_group_34 = receiver_group_34.astype({'rallyCount': 'int64'})


# n, x, p(=x/n) 값 구하기

n_1 = len(receiver_group_1[receiver_group_1['rallyCount']!=0])
n_2 = len(receiver_group_2[receiver_group_2['rallyCount']!=0])
n_3 = len(receiver_group_3[receiver_group_3['rallyCount']!=0])
n_4 = len(receiver_group_4[receiver_group_4['rallyCount']!=0])
n_12 = len(receiver_group_12[receiver_group_12['rallyCount']!=0])
n_34 = len(receiver_group_34[receiver_group_34['rallyCount']!=0])

x_1 = len(receiver_group_1[(receiver_group_1['rallyCount']!=0) & (receiver_group_1['isRcvrWinner']==1)])
x_2 = len(receiver_group_2[(receiver_group_2['rallyCount']!=0) & (receiver_group_2['isRcvrWinner']==1)])
x_3 = len(receiver_group_3[(receiver_group_3['rallyCount']!=0) & (receiver_group_3['isRcvrWinner']==1)])
x_4 = len(receiver_group_4[(receiver_group_4['rallyCount']!=0) & (receiver_group_4['isRcvrWinner']==1)])
x_12 = len(receiver_group_12[(receiver_group_12['rallyCount']!=0) & (receiver_group_12['isRcvrWinner']==1)])
x_34 = len(receiver_group_34[(receiver_group_34['rallyCount']!=0) & (receiver_group_34['isRcvrWinner']==1)])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34



In [ ]:
p_1

0.2975875743555849

In [ ]:
p_2

0.3464639061927446

In [ ]:
p_12

0.3207645525629887

In [ ]:
p_3

0.34950819672131145

In [ ]:
p_4

0.3716171617161716

In [ ]:
p_34

0.3642151481888035

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 4555
게임포인트의 n: 11510
 
브레이크포인트의 x: 1659
게임포인트의 x: 3692
 
브레이크포인트의 p: 0.3642151481888035
게임포인트의 p: 0.3207645525629887
 
z_score: 5.202
p_value: 9.855847327155942e-08
 
가설 채택


# 1st in

### Server용

In [ ]:
server_group_1

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount
8,'Hubert_Hurkacz',14,0,0,0,2.0,40-0,3 (4),1,0,...,1.0,False,False,False,False,True,False,1,1,1
12,'Hubert_Hurkacz',24,0,0,1,3.0,40-0,5 (4),1,0,...,0.0,False,False,False,False,True,False,2,0,2
13,'Hubert_Hurkacz',25,0,0,1,3.0,40-15,5 (5),1,0,...,0.0,False,False,False,True,False,False,2,0,8
19,'Hubert_Hurkacz',37,0,0,2,4.0,40-15,7 (5),1,0,...,0.0,True,False,False,False,False,False,1,1,1
23,'Hubert_Hurkacz',49,0,0,3,5.0,40-0,9 (4),1,0,...,0.0,False,False,True,False,False,False,2,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61399,'Fernando_Gonzalez',188,1,2,2,1.0,40-0,32 (4),1,0,...,0.0,True,False,False,False,False,False,2,1,1.0
61433,'Fernando_Gonzalez',241,1,2,6,5.0,40-0,40 (4),1,0,...,1.0,False,False,False,True,False,False,1,0,2.0
61434,'Fernando_Gonzalez',242,1,2,6,5.0,40-15,40 (5),1,0,...,0.0,False,False,True,False,False,False,1,0,8.0
61447,'Fernando_Gonzalez',266,2,2,3,0.0,40-0,44 (4),0,0,...,0.0,False,False,True,False,False,False,1,0,4.0


In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(server_group_1)
n_2 = len(server_group_2)
n_3 = len(server_group_3)
n_4 = len(server_group_4)
n_12 = len(server_group_12)
n_34 = len(server_group_34)

x_1 = len(server_group_1[server_group_1['1stIn']==1])
x_2 = len(server_group_2[server_group_2['1stIn']==1])
x_3 = len(server_group_3[server_group_3['1stIn']==1])
x_4 = len(server_group_4[server_group_4['1stIn']==1])
x_12 = len(server_group_12[server_group_12['1stIn']==1])
x_34 = len(server_group_34[server_group_34['1stIn']==1])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
print(n_1)
print(x_1)

6304
3862


In [ ]:
p_1

0.6126269035532995

In [ ]:
p_2

0.6266216456371068

In [ ]:
p_12

0.6192272232000671

In [ ]:
p_3

0.6285896617740906

In [ ]:
p_4

0.61376

In [ ]:
p_34

0.6187127024722933

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 4692
게임포인트의 n: 11931
 
브레이크포인트의 x: 2903
게임포인트의 x: 7388
 
브레이크포인트의 p: 0.6187127024722933
게임포인트의 p: 0.6192272232000671
 
z_score: -0.061
p_value: 0.4754888732352297
 
가설 기각


# is Received

### Receiver

In [ ]:
receiver_group_1 = receiver_group_1.astype({'rallyCount': 'int64'})
receiver_group_2 = receiver_group_2.astype({'rallyCount': 'int64'})
receiver_group_12 = receiver_group_12.astype({'rallyCount': 'int64'})

receiver_group_3 = receiver_group_3.astype({'rallyCount': 'int64'})
receiver_group_4 = receiver_group_4.astype({'rallyCount': 'int64'})
receiver_group_34 = receiver_group_34.astype({'rallyCount': 'int64'})

In [ ]:
receiver_group_1['isReceived'] = receiver_group_1['rallyCount'].apply(lambda x: '0' if (x == 0) else '1')
receiver_group_2['isReceived'] = receiver_group_2['rallyCount'].apply(lambda x: '0' if (x == 0) else '1')
receiver_group_3['isReceived'] = receiver_group_3['rallyCount'].apply(lambda x: '0' if (x == 0) else '1')
receiver_group_4['isReceived'] = receiver_group_4['rallyCount'].apply(lambda x: '0' if (x == 0) else '1')
receiver_group_12['isReceived'] = receiver_group_12['rallyCount'].apply(lambda x: '0' if (x == 0) else '1')
receiver_group_34['isReceived'] = receiver_group_34['rallyCount'].apply(lambda x: '0' if (x == 0) else '1')

receiver_group_1 = receiver_group_1.astype({'isReceived': 'int64'})
receiver_group_2 = receiver_group_2.astype({'isReceived': 'int64'})
receiver_group_12 = receiver_group_12.astype({'isReceived': 'int64'})

receiver_group_3 = receiver_group_3.astype({'isReceived': 'int64'})
receiver_group_4 = receiver_group_4.astype({'isReceived': 'int64'})
receiver_group_34 = receiver_group_34.astype({'isReceived': 'int64'})

In [ ]:
# n, x, p(=x/n) 값 구하기

n_1 = len(receiver_group_1)
n_2 = len(receiver_group_2)
n_3 = len(receiver_group_3)
n_4 = len(receiver_group_4)
n_12 = len(receiver_group_12)
n_34 = len(receiver_group_34)

x_1 = len(receiver_group_1[receiver_group_1['isReceived']==1])
x_2 = len(receiver_group_2[receiver_group_2['isReceived']==1])
x_3 = len(receiver_group_3[receiver_group_3['isReceived']==1])
x_4 = len(receiver_group_4[receiver_group_4['isReceived']==1])
x_12 = len(receiver_group_12[receiver_group_12['isReceived']==1])
x_34 = len(receiver_group_34[receiver_group_34['isReceived']==1])

p_1 = x_1/n_1
p_2 = x_2/n_2
p_3 = x_3/n_3
p_4 = x_4/n_4
p_12 = x_12/n_12
p_34 = x_34/n_34

In [ ]:
p_1

0.9600253807106599

In [ ]:
p_2

0.9699662342278301

In [ ]:
p_12

0.9647137708490487

In [ ]:
p_3

0.97319719208679

In [ ]:
p_4

0.9696

In [ ]:
p_34

0.9708013640238704

## 브레이크포인트 상황 - 게임포인트 상황

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import norm

print("브레이크포인트의 n:", n_34)
print("게임포인트의 n:", n_12)
print(" ")
print("브레이크포인트의 x:", x_34)
print("게임포인트의 x:", x_12)
print(" ")
print("브레이크포인트의 p:", p_34)
print("게임포인트의 p:", p_12)
print(" ")

SE = np.sqrt(p_34 * (1 - p_34) / n_34 + p_12 * (1 - p_12) / n_12)
z_score = (p_34 - p_12) / SE  # z score: z값 부호에 따라 p1과 p2중 뭐가 큰지 결과가 나옴
p_value = 1 - norm.cdf(abs(z_score))  # p value: p1 != p2(귀무가설)에 대해 유의미한 결과인지 확인

print("z_score:", z_score.round(3))
print("p_value:", p_value)
print(" ")

if p_value < 0.05:
  print("가설 채택")
else:
  print("가설 기각")

브레이크포인트의 n: 4692
게임포인트의 n: 11931
 
브레이크포인트의 x: 4555
게임포인트의 x: 11510
 
브레이크포인트의 p: 0.9708013640238704
게임포인트의 p: 0.9647137708490487
 
z_score: 2.041
p_value: 0.020615801093419095
 
가설 채택
